In [ ]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF = 1
max_attempts = 2
TIMEOUT_SECONDS = 180  
output_dir="/tmp/llm"
model1 = "o3-mini"
model2 = "qwen3:8b"

In [ ]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os
import threading


# Generate Data

In [ ]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [ ]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [ ]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        con.sql("SET enable_progress_bar_print = false")
        con.sql("SET progress_bar_time = 0")

        result_container = {"result": None, "error": None}

        def query_thread():
            try:
                result_container["result"] = con.execute(current_query).fetchdf()
            except duckdb.InterruptException:
                result_container["error"] = f"Query interrupted after timeout of {TIMEOUT_SECONDS} seconds."
            except Exception as e:
                result_container["error"] = str(e)

        thread = threading.Thread(target=query_thread)
        thread.start()

        start_time = time.time()
        while thread.is_alive():
            elapsed = time.time() - start_time
            if elapsed > TIMEOUT_SECONDS:
                con.interrupt()
                thread.join()
                return f"Query execution timed out after {TIMEOUT_SECONDS} seconds.", attempt, "query runs forever"
            time.sleep(0.1)

        if result_container["error"]:
            error_message = result_container["error"].lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {result_container['error']}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {result_container['error']}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {result_container['error']}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {result_container['error']}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1
        else:
            return result_container["result"], attempt, current_query

    return "Unexpected error or loop termination", attempt, current_query


In [ ]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })

    os.makedirs(output_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
            f.flush()
            os.fsync(f.fileno())
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [ ]:
# get the questions from GitHub
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

In [ ]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

In [ ]:
%%time
ask_question(questions,model2,"ollama")

# Check Results

In [15]:
duckdb.sql(f""" select *,cardinality(result[1]) as nbr_columns, [result_count , nbr_columns] as result_shape from '{output_dir}/*.json'  """).to_view("results")
try:
 duckdb.sql(f""" install excel ; load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           with zzzz as (select nbr,question,model,result_shape from results )
           pivot( select nbr,question,model,list(distinct(result_shape)) as result_shape from zzzz group by all) 
           on model using min(result_shape) order by nbr
        """).show(max_width=120)

Error exporting to Excel: IO Error: Could not move file: The process cannot access the file because it is being used by another process.

close the excel file or tey again.
┌───────┬──────────────────────┬──────────────────────┬──────────────┬──────────────────────┬──────────────────────────┐
│  nbr  │       question       │       o3-mini        │  qwen3:1.7b  │       qwen3:4b       │         qwen3:8b         │
│ int64 │       varchar        │      int128[][]      │  int128[][]  │      int128[][]      │        int128[][]        │
├───────┼──────────────────────┼──────────────────────┼──────────────┼──────────────────────┼──────────────────────────┤
│     1 │ What is the overal…  │ [[1, 1]]             │ [[0, NULL]]  │ [[1, 1]]             │ [[1, 1]]                 │
│     2 │ What is the total …  │ [[1, 1]]             │ [[1, 1]]     │ [[1, 1]]             │ [[1, 1], [0, NULL]]      │
│     3 │ What is the total …  │ [[1, 1]]             │ [[0, NULL]]  │ [[1, 1]]             │ [[1, 1]

compare the latest runs

In [16]:
duckdb.sql(f""" 
        create or replace temp table results_filtered as
           from results  where model = '{model1}' and timestamp = (select max(timestamp) from results where model = '{model1}') 
           union all 
           from results  where model = '{model2}' and timestamp = (select max(timestamp) from results where model = '{model2}')
""")
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr as question ,model,result_shape , attempts from results_filtered ) 
           on model using min(result_shape) as result,  min(attempts) as attempts order by question
        """).show(max_width=130)

┌──────────┬────────────────┬──────────────────┬─────────────────┬───────────────────┐
│ question │ o3-mini_result │ o3-mini_attempts │ qwen3:8b_result │ qwen3:8b_attempts │
│  int64   │    int128[]    │      int64       │    int128[]     │       int64       │
├──────────┼────────────────┼──────────────────┼─────────────────┼───────────────────┤
│        1 │ [1, 1]         │                1 │ [1, 1]          │                 1 │
│        2 │ [1, 1]         │                1 │ [1, 1]          │                 2 │
│        3 │ [1, 1]         │                1 │ [1, 1]          │                 1 │
│        4 │ [12, 1]        │                1 │ [12, 1]         │                 1 │
│        5 │ [11, 1]        │                1 │ [11, 1]         │                 1 │
│        6 │ [6, 2]         │                1 │ [6, 2]          │                 1 │
│        7 │ [1, 2]         │                1 │ [6, 2]          │                 1 │
│        8 │ [713, 2]       │              

In [17]:
def display_side_by_side(nbr, model1, model2):
    print(f"question {nbr} : " + duckdb.sql(f" select question from results_filtered where nbr = {nbr}  ").fetchone()[0])
    try:
        df1 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model1}' """).fetchone()[0]).df()        
        df2 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model2}' """).fetchone()[0]).df() 
        side_by_side = pd.concat([df1, df2], axis=1, keys=[model1, model2])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

In [27]:
different_results = duckdb.sql(f"""
    SELECT 
        distinct(nbr) 
    FROM results_filtered 
    WHERE nbr in
    (
    select nbr from results_filtered 
    group by all
    having count(distinct(nbr_columns)) >  1 or count(distinct(result_count )) > 1 
    )
    order by nbr
""").df()['nbr'].tolist()
print(f"Different results for {len(different_results)} questions: {different_results}")
good_results = list(set(list(range(1, 21))) - set(different_results))
print(f"Similar resultsets size for {len(good_results)} questions: {good_results}")


Different results for 7 questions: [7, 14, 15, 16, 17, 18, 20]
Similar resultsets size for 13 questions: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 19]


In [30]:
for nbr in list(range(1, 21)):
    display_side_by_side(nbr, model1, model2)

question 1 : What is the overall total sales revenue?


,o3-mini,qwen3:8b
,total_sales_revenue,total_sales
0,5.138666e+09,5.138666e+09


question 2 : What is the total number of items sold across all transactions?


,o3-mini,qwen3:8b
,total_quantity,total_quantity
0,138963631.0,138963631.0


question 3 : What is the total monetary value of all returned items?


...

question 17 : What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45), ordered by age group.


o3-mini                                                    qwen3:8b  \
   sale_day_type age_group   total_sales total_returns return_rate age_group   
0        Weekday  Under 30  1.106583e+09   56395803.78    5.096389     20-29   
1        Weekday     30-45  8.036384e+08   42231793.68    5.255074     20-29   
2        Weekday   Over 45  1.568486e+09   83036647.81    5.294065     30-39   
3        Weekend  Under 30  4.411807e+08   23060101.47    5.226906     30-39   
4        Weekend     30-45  3.256532e+08   16958203.53    5.207442     40-49   
5        Weekend   Over 45  6.254228e+08   32936328.13    5.266250     40-49   
6            NaN       NaN           NaN           NaN         NaN     50-59   
7            NaN       NaN           NaN           NaN         NaN     50-59   
8            NaN       NaN           NaN           NaN         NaN       60+   
9            NaN       NaN           NaN           NaN         NaN       60+   
10           NaN       NaN           NaN           NaN         NaN      < 20   
11           NaN       NaN           NaN           NaN         NaN      < 20   

                                                      
   d_weekend   total_sales total_returns return_rate  
0          N  5.098070e+08   26393168.04    5.177090  
1          Y  2.025206e+08   11042123.56    5.452347  
2          N  5.036592e+08   26550551.04    5.271531  
3          Y  2.044319e+08   10821169.42    5.293287  
4          N  5.053419e+08   26399997.04    5.224185  
5          Y  2.025687e+08   10441801.82    5.154697  
6          Y  2.029946e+08   10674998.32    5.258759  
7          N  5.058094e+08   26516422.79    5.242374  
8          N  8.573134e+08   45801470.62    5.342442  
9          Y  3.410808e+08   17956562.10    5.264606  
10         N  5.967765e+08   30002635.74    5.027449  
11         Y  2.386602e+08   12017977.91    5.035602

question 18 : Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).
Error executing query for nbr 18: Binder Error: Values list "br1" does not have a column named "avg_return_rate_2002"
question 19 : For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.


o3-mini                                    qwen3:8b                \
       i_class preferred_customer avg_net_sales item_class customer_type   
0    wallpaper               None   2182.882320  wallpaper          None   
1        decor               None   2172.952359      decor          None   
2       stereo               None   2145.389483     stereo          None   
3      fishing               None   2100.967304    fishing          None   
4       custom               None   2078.424679     custom          None   
..         ...                ...           ...        ...           ...   
295   flatware               None   1757.351243   flatware          None   
296    musical               None   1756.913914    musical          None   
297     sports               None   1747.283095     sports          None   
298    camping               None   1741.471667    camping          None   
299     tennis               None   1734.447058     tennis          None   

                                   
    avg_net_sales_per_transaction  
0                     2182.882320  
1                     2172.952359  
2                     2145.389483  
3                     2100.967304  
4                     2078.424679  
..                            ...  
295                   1757.351243  
296                   1756.913914  
297                   1747.283095  
298                   1741.471667  
299                   1734.447058  

[300 rows x 6 columns]

question 20 : show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
Error executing query for nbr 20: Binder Error: Referenced table "sna" not found!
Candidate tables: "sns"
